In [1]:
import pandas as pd

In [2]:
pd.__version__

'1.2.4'

## Assess SRA Search Results

#### Load Results

In [3]:
%%bash

mkdir -p ../output.gubaphage-magsearch/results
curl -L https://osf.io/x2fwy/download -o ../output.gubaphage-magsearch/results/gubaphage.k21.csv
ls ../output.gubaphage-magsearch/results

gubaphage.k21.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   513  100   513    0     0    168      0  0:00:03  0:00:03 --:--:--   168
100 6326k  100 6326k    0     0   945k      0  0:00:06  0:00:06 --:--:-- 2710k


In [4]:
# read in k=21 SRA search results
ms21 = pd.read_csv("../output.gubaphage-magsearch/results/gubaphage.k21.csv",
                   sep=",",
                   quotechar="'",
                   header=0,
                   names=["search_genome", "metagenome", "containment"])
ms21

,search_genome,metagenome,containment
0,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.027231
1,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.012103
2,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.040217
3,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.021684
4,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.010590
...,...,...,...
56993,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.018659
56994,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.013490
56995,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.031518
56996,Gubaphage_genomes.fa,/group/ctbrowngrp/irber/data/wort-data/wort-sr...,0.012733


In [5]:
# Fix names so it's easier to query
ms21['search_genome'] = ms21['search_genome'].str.replace(r"'(?P<id>.*)'", lambda m: m.group("id"), regex=True)
ms21['metagenome'] = ms21['metagenome'].str.replace(r".*/(?P<id>.*).sig.*", lambda m: m.group("id"), regex=True)
ms21['accession'] = ms21['search_genome'].str.split(" ", 1, expand=True)[0]
ms21

,search_genome,metagenome,containment,accession
0,Gubaphage_genomes.fa,ERR1518322,0.027231,Gubaphage_genomes.fa
1,Gubaphage_genomes.fa,ERR4090756,0.012103,Gubaphage_genomes.fa
2,Gubaphage_genomes.fa,SRR6747964,0.040217,Gubaphage_genomes.fa
3,Gubaphage_genomes.fa,SRR5580203,0.021684,Gubaphage_genomes.fa
4,Gubaphage_genomes.fa,ERR4564803,0.010590,Gubaphage_genomes.fa
...,...,...,...,...
56993,Gubaphage_genomes.fa,SRR7723106,0.018659,Gubaphage_genomes.fa
56994,Gubaphage_genomes.fa,SRR8675863,0.013490,Gubaphage_genomes.fa
56995,Gubaphage_genomes.fa,SRR8451930,0.031518,Gubaphage_genomes.fa
56996,Gubaphage_genomes.fa,SRR12682966,0.012733,Gubaphage_genomes.fa


In [6]:
# print some summary info for 50%+ containment
search_ksize = 21
num_search_genomes = len(ms21["search_genome"].unique())
num_search_genomes_with_matches = len(ms21[ms21['containment'] > 0.5]["search_genome"].unique())
num_unique_metagenome_matches = len(ms21[ms21['containment'] > 0.5]["metagenome"].unique())

print(f"Search ksize: {search_ksize}")
print(f"# Search genomes: {num_search_genomes}")
print(f"# Search genomes with 'good' metagenome matches: {num_search_genomes_with_matches}")
print(f"# Metagenomes with >50% containment of at least one search genome: {num_unique_metagenome_matches}")

Search ksize: 21
# Search genomes: 1
# Search genomes with 'good' metagenome matches: 1
# Metagenomes with >50% containment of at least one search genome: 1


In [7]:
# Summarize 30% containment
search_ksize = 21
c30_matches = len(ms21[ms21['containment'] > 0.3]["metagenome"].unique())
print(f"Search ksize: {search_ksize}")
print(f"# Metagenomes with >30% containment of at least one search genome: {c30_matches}")

Search ksize: 21
# Metagenomes with >30% containment of at least one search genome: 94


In [8]:
# Summarize 10% containment (should be all results)
search_ksize = 21
c10_matches = len(ms21[ms21['containment'] > 0.1]["metagenome"].unique())
print(f"Search ksize: {search_ksize}")
print(f"# Metagenomes with >10% containment of at least one search genome: {c10_matches}")

Search ksize: 21
# Metagenomes with >10% containment of at least one search genome: 889


## Explore & Summarize >30% containment results

In [9]:
#subset to just >30% containment; sort by containment & look at top metagenome matches
ms21_c30 = ms21[ms21['containment'] > 0.3]
ms21_c30 = ms21_c30.sort_values(by="containment", ascending=False, ignore_index=True)
ms21_c30

,search_genome,metagenome,containment,accession
0,Gubaphage_genomes.fa,ERR2683220,0.505547,Gubaphage_genomes.fa
1,Gubaphage_genomes.fa,ERR2683247,0.471886,Gubaphage_genomes.fa
2,Gubaphage_genomes.fa,ERR2592250,0.438855,Gubaphage_genomes.fa
3,Gubaphage_genomes.fa,ERR2607412,0.438855,Gubaphage_genomes.fa
4,Gubaphage_genomes.fa,ERR2683203,0.431543,Gubaphage_genomes.fa
...,...,...,...,...
89,Gubaphage_genomes.fa,ERR2683244,0.303076,Gubaphage_genomes.fa
90,Gubaphage_genomes.fa,ERR2607430,0.301311,Gubaphage_genomes.fa
91,Gubaphage_genomes.fa,ERR1713347,0.301311,Gubaphage_genomes.fa
92,Gubaphage_genomes.fa,ERR2683266,0.300177,Gubaphage_genomes.fa


In [10]:
# save sorted containment-50 csvs
!mkdir -p ../output.gubaphage-magsearch/processed_results

ms21_c30.to_csv("../output.gubaphage-magsearch/processed_results/gubaphage.sra-search.k21-c30.csv", index=False)

In [11]:
# save list of all unique matched metagenomes
m21_metagenomes = ms21_c30["metagenome"].unique()
!mkdir -p ../output.magsearch/processed_results
with open('../output.gubaphage-magsearch/processed_results/ms21_c30.unique-metagenomes.txt', 'w') as out:
    for mg in list(m21_metagenomes):
        out.write(f"{mg}\n")
m21_metagenomes

array(['ERR2683220', 'ERR2683247', 'ERR2592250', 'ERR2607412',
       'ERR2683203', 'ERR2683256', 'ERR2683152', 'ERR1713391',
       'ERR2607540', 'ERR2200456', 'ERR2683275', 'ERR2683216',
       'ERR2683150', 'ERR2683157', 'ERR2607536', 'ERR2683176',
       'ERR1713390', 'ERR2683228', 'ERR2683138', 'ERR2683209',
       'ERR2607387', 'ERR1713334', 'ERR2683143', 'ERR2683251',
       'ERR1713372', 'ERR2607487', 'ERR2683263', 'ERR2683236',
       'ERR2607386', 'ERR2592246', 'ERR1713350', 'ERR2607437',
       'ERR1713388', 'ERR2607531', 'ERR2683153', 'ERR2683272',
       'ERR2683137', 'ERR2683198', 'ERR2607497', 'ERR2592263',
       'ERR2683177', 'ERR2607498', 'ERR1713375', 'ERR2683261',
       'ERR2683149', 'SRR11088412', 'ERR2683235', 'ERR2683219',
       'ERR1713389', 'ERR2607533', 'ERR2683221', 'ERR2683233',
       'ERR2683227', 'ERR2683259', 'ERR2683142', 'SRR11088470',
       'ERR2592329', 'ERR2607403', 'ERR2683252', 'ERR3563106',
       'ERR2683222', 'ERR2683234', 'ERR2683230', 'ERR

## Explore & Summarize >10% containment results

In [12]:
#subset to just >30% containment; sort by containment & look at top metagenome matches
ms21_c10 = ms21[ms21['containment'] > 0.1]
ms21_c10 = ms21_c10.sort_values(by="containment", ascending=False, ignore_index=True)
ms21_c10

,search_genome,metagenome,containment,accession
0,Gubaphage_genomes.fa,ERR2683220,0.505547,Gubaphage_genomes.fa
1,Gubaphage_genomes.fa,ERR2683247,0.471886,Gubaphage_genomes.fa
2,Gubaphage_genomes.fa,ERR2607412,0.438855,Gubaphage_genomes.fa
3,Gubaphage_genomes.fa,ERR2592250,0.438855,Gubaphage_genomes.fa
4,Gubaphage_genomes.fa,ERR2683203,0.431543,Gubaphage_genomes.fa
...,...,...,...,...
884,Gubaphage_genomes.fa,ERR2298114,0.100227,Gubaphage_genomes.fa
885,Gubaphage_genomes.fa,SRR6429805,0.100227,Gubaphage_genomes.fa
886,Gubaphage_genomes.fa,ERR3502017,0.100101,Gubaphage_genomes.fa
887,Gubaphage_genomes.fa,SRR4295172,0.100101,Gubaphage_genomes.fa


In [13]:
ms21_c10 = ms21_c10.sort_values(by="containment", ascending=False, ignore_index=True)
ms21_c10.to_csv("../output.gubaphage-magsearch/processed_results/gubaphage.sra-search.k21.csv", index=False)

In [14]:
# save list of all unique matched metagenomes
m21_c10_metagenomes = ms21_c10["metagenome"].unique()
!mkdir -p ../output.magsearch/processed_results
with open('../output.gubaphage-magsearch/processed_results/ms21.unique-metagenomes.txt', 'w') as out:
    for mg in list(m21_c10_metagenomes):
        out.write(f"{mg}\n")
m21_c10_metagenomes

array(['ERR2683220', 'ERR2683247', 'ERR2607412', 'ERR2592250',
       'ERR2683203', 'ERR2683256', 'ERR2683152', 'ERR2607540',
       'ERR1713391', 'ERR2200456', 'ERR2683275', 'ERR2683216',
       'ERR2683150', 'ERR2683157', 'ERR2607536', 'ERR2683176',
       'ERR1713390', 'ERR2683228', 'ERR2683138', 'ERR2683209',
       'ERR1713334', 'ERR2607387', 'ERR2683143', 'ERR2683251',
       'ERR1713372', 'ERR2607487', 'ERR2683263', 'ERR2683236',
       'ERR2592246', 'ERR2607386', 'ERR2607437', 'ERR1713350',
       'ERR1713388', 'ERR2607531', 'ERR2683153', 'ERR2683272',
       'ERR2683137', 'ERR2683198', 'ERR2607497', 'ERR2592263',
       'ERR2683177', 'ERR1713375', 'ERR2607498', 'ERR2683261',
       'ERR2683149', 'SRR11088412', 'ERR2683235', 'ERR2683219',
       'ERR2607533', 'ERR1713389', 'ERR2683221', 'ERR2683233',
       'ERR2683227', 'ERR2683259', 'ERR2683142', 'SRR11088470',
       'ERR2607403', 'ERR2592329', 'ERR2683252', 'ERR3563106',
       'ERR2683234', 'ERR2683222', 'ERR2683230', 'ERR

In [15]:
len(m21_c10_metagenomes)

889